# Capture historic polling data from Wikipedia

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Output-location" data-toc-modified-id="Output-location-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Output location</a></span></li><li><span><a href="#Raw-data-capture" data-toc-modified-id="Raw-data-capture-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Raw data capture</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Compile-a-table-of-opinion-polls" data-toc-modified-id="Compile-a-table-of-opinion-polls-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compile a table of opinion polls</a></span></li><li><span><a href="#Save" data-toc-modified-id="Save-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save</a></span></li></ul></div>

## Python set-up

In [1]:
# data science imports
import pandas as pd

# system imports
import pathlib

# local imports
import common

## Output location

In [2]:
DATA_DIR = '../historical-data'
pathlib.Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
DATA_DIR += '/'

## Raw data capture

In [3]:
links = {
    # election-date: [url, table-number],
    '2019-05-18': ['https://en.wikipedia.org/wiki/'
                   'Opinion_polling_for_the_2019_Australian_federal_election',
                   1],
    '2016-07-02': ['https://en.wikipedia.org/wiki/'
                   'National_opinion_polling_for_the_2016_Australian_federal_election',
                   2],
    '2013-09-07': ['https://en.wikipedia.org/wiki/'
                   'Opinion_polling_for_the_2013_Australian_federal_election',
                   1],
    
    # This is only Newspoll polling data ... also oddly formatted
    #'2010-08-21': ['https://en.wikipedia.org/wiki/'
    #               'Opinion_polling_for_the_2010_Australian_federal_election',
    #               0],
    
    # Earlier polling data does not appear to be available on Wikipedia
}

In [4]:
def capture_raw_tables(links):
    raw_tables = {}
    for elect_date, (url, number) in links.items():
        text = common.get_url_text(url)
        table = common.get_table_from_text(number, text)
        raw_tables[elect_date] = table
    return raw_tables

raw_tables = capture_raw_tables(links)

AttributeError: module 'common' has no attribute 'get_url_text'

In [ ]:
if False: # display tables
    for elect_date, table in raw_tables.items():
        print(elect_date)
        display(table)

## Data cleaning

In [ ]:
clean_tables = {}
for election_date, table in raw_tables.items():
    clean_tables[election_date] = common.clean(table)

## Compile a table of opinion polls 

using data for the period 3 months prior to a Federal election

In [ ]:
def compile_table(clean_tables):

    RELEVANT_PERIOD = 100 # DAYS
    keep_cols = ['Date', 'Firm', 'Primary vote L/NP', 
                 'Primary vote ALP',
                 'Primary vote GRN', 'Primary vote ONP', 
                 'Primary vote OTH',
                 '2PP vote L/NP', '2PP vote ALP']

    final_table = None
    for election, table in clean_tables.items():
    
        # standardise column names
        table = table.copy().rename(columns={'TPP vote': '2PP vote'}, level=0)
        table = table.rename(columns={'L/NP*': 'L/NP'}, level=1)
        table = table.rename(columns={'Labor': 'ALP'}, level=1)
        table = table.rename(columns={'Green': 'GRN'}, level=1)
        table = table.rename(columns={'Other': 'OTH'}, level=1)
        table.columns = common.flatten_col_names(table.columns)
    
        # select most recent rows
        keep = pd.Timestamp(election) - pd.Timedelta(days=RELEVANT_PERIOD)
        table = table[table.index >= keep]
        keepers = [x for x in keep_cols if x in table.columns]
        table = table[keepers]
        table['Election'] = election
    
        # merge into a single table we will save ...
        final_table = (
            # There must be an easier way of doing this ... 
            table if final_table is None 
            else final_table.T.merge(table.T, how='outer', 
                                     left_index=True, 
                                     right_index=True).T
        )
        
    return final_table


final_table = compile_table(clean_tables)

## Save

In [ ]:
final_table.to_excel(DATA_DIR+'historical-poll-data.xlsx')